In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 105)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
sns.set()

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore")

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [3]:
nr_cv = 5

use_logvalus = 1

target = "SalePrice_Log"

min_val_corr = 0.4

drop_similar = 1

In [4]:
def get_best_score(grid):
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)
    print(grid.best_params_)
    print(grid.best_estimator_)
    
    return best_score

In [5]:
def print_cols_large_corr(df, nr_c, targ):
    corr = df.corr()
    corr_abs = corr.abs()
    print(corr_abs.nlargest(nr_c, targ)[targ])

In [6]:
def plot_corr_matlix(df, nr_c, targ):
    corr = df.corr()
    corr_abs = corr.abs()
    cols = corr_abs.nlargest(nr_c, targ)[targ].index
    cm = np.corrcoef(df[cols].values.T)
    
    plt.figure(figsize=(nc_c/1.5, nr_c/1.5))
    sns.set(font_scale=1.25)
    sns.heatmap(cm, linewidth=1.5, annot=True, square=True,
               fmt=".2f", annot_kws={"size":10},
               yticklabels=cols.values, xticklabels=cols.value)
    plt.show()

In [7]:
df_train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [8]:
print(df_train.shape)
print("*"*50)
print(df_test.shape)

In [9]:
print(df_train.info())
print("*"*50)
print(df_test.info())

In [10]:
df_train.head()

In [11]:
df_train.describe()

In [12]:
df_test.head()

In [13]:
df_test.describe()

In [14]:
sns.distplot(df_train["SalePrice"])

print("Skewness: %f" % df_train["SalePrice"].skew())
print("Kurtosis: %f" % df_train["SalePrice"].kurt())

In [15]:
df_train["SalePrice_Log"] = np.log(df_train["SalePrice"])

sns.distplot(df_train["SalePrice_Log"])

print("Skewness: %f" % df_train["SalePrice_Log"].skew())
print("Kurtosis: %f" % df_train["SalePrice_Log"].kurt())

df_train.drop("SalePrice", axis=1, inplace=True)

In [16]:
numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index

print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = df_train.dtypes[df_train.dtypes == "object"].index

print("Number of Categorical features: ", len(categorical_feats))

In [17]:
print(df_train[numerical_feats].columns)
print("*"*100)
print(df_train[categorical_feats].columns)

In [18]:
df_train[numerical_feats].head()

In [19]:
df_train[categorical_feats].head()

In [20]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=["Total", "Percent"])
missing_data.head(20)

In [21]:
# columns where NaN values have meaning e.g. no pool etc.
cols_fillna = ['PoolQC','MiscFeature','Alley','Fence','MasVnrType','FireplaceQu',
               'GarageQual','GarageCond','GarageFinish','GarageType', 'Electrical',
               'KitchenQual', 'SaleType', 'Functional', 'Exterior2nd', 'Exterior1st',
               'BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2',
               'MSZoning', 'Utilities']

for col in cols_fillna:
    df_train[col].fillna("None", inplace=True)
    df_test[col].fillna("None", inplace=True)

In [22]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum() / df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=["Total", "Percent"])
missing_data.head(5)

In [23]:
df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)

In [24]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(5)

In [25]:
df_train.isnull().sum().sum()

In [26]:
df_test.isnull().sum().sum()

In [27]:
for col in numerical_feats:
    print("{:15}".format(col),
          "Skewness: {:05.2f}".format(df_train[col].skew()),
          "    ",
          "Kurtosis: {:06.2f}".format(df_train[col].kurt()))

In [28]:
sns.distplot(df_train["GrLivArea"])

print("Skewness: %f" % df_train["GrLivArea"].skew())
print("Kurtosis: %f" % df_train["GrLivArea"].kurt())

In [29]:
sns.distplot(df_train["LotArea"])

print("Skewness: %f" % df_train["LotArea"].skew())
print("Kurtosis: %f" % df_train["LotArea"].kurt())

In [30]:
for df in [df_train, df_test]:
    df["GrLivArea_Log"] = np.log(df["GrLivArea"])
    df.drop("GrLivArea", inplace=True, axis=1)
    df["LotArea_Log"] = np.log(df["LotArea"])
    df.drop("LotArea", inplace=True, axis=1)
    
numerical_feats = df_train.dtypes[df_train.dtypes != "object"].index

In [31]:
sns.distplot(df_train["GrLivArea_Log"])

print("Skewness : ", df_train["GrLivArea_Log"].skew())
print("Kurtosis : ", df_train["GrLivArea_Log"].kurt())

In [32]:
sns.distplot(df_train["LotArea_Log"])

print("Skewness : ", df_train["LotArea_Log"].skew())
print("Kurtsis : ", df_train["LotArea_Log"].kurt())

In [33]:
nr_rows = 12
nr_cols = 3

fig, ax = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*3.5, nr_rows*3))

li_num_feats = list(numerical_feats)
li_not_plot = ["Id", "SalePrice", "SalePrice_Log"]
li_plot_num_feats = [c for c in list(numerical_feats) if c not in li_not_plot]

for r in range(0, nr_rows):
    for c in range(0, nr_cols):
        i = r * nr_cols + c
        if i < len(li_plot_num_feats):
            sns.regplot(df_train[li_plot_num_feats[i]],
                        df_train[target],
                        ax = ax[r][c])
            stp = stats.pearsonr(df_train[li_plot_num_feats[i]],
                                 df_train[target])
            str_title = "r = " + "{0:.2f}".format(stp[0]) + "  " + "p = " + "{0:.2f}".format(stp[1])
            ax[r][c].set_title(str_title, fontsize=11)
            
plt.tight_layout()
plt.show()